In [ ]:
!pip install python-docx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 2.2 MB/s eta 0:00:00


In [ ]:
!pip install openai faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 743.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain_openai

In [ ]:
import os
import tiktoken
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
import docx

class OpenAIChat:
    def __init__(self, api_key: str, model_name: str = "gpt-4", temperature: float = 0.7):
        os.environ["OPENAI_API_KEY"] = api_key
        self.llm = ChatOpenAI(temperature=temperature, model_name=model_name)
        self.memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
        self.vectorstore = None

    def load_docx(self, file_path: str):
        doc = docx.Document(file_path)
        full_text = []
        for para in doc.paragraphs:
            full_text.append(para.text)
        self.data = "\n".join(full_text)

    def split_text(self, chunk_size: int = 1000, chunk_overlap: int = 200):
        text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        document = Document(page_content=self.data)
        self.data = text_splitter.split_documents([document])

    def create_vectorstore(self):
        embeddings = OpenAIEmbeddings()
        self.vectorstore = FAISS.from_documents(self.data, embedding=embeddings)

    def create_conversation_chain(self):
        self.conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(),
            memory=self.memory
        )

    def ask_question(self, query: str) -> str:
        result = self.conversation_chain({"question": query})
        return result["answer"]



In [ ]:
import docx
from langchain.schema import Document as LangChainDocument  # Rename for clarity

# Rest of your class code remains the same...

# Adjust your function to explicitly use docx.Document
def store_response_in_docx(response_text, output_path):
    doc = docx.Document()  # Create a new Document using docx
    doc.add_paragraph(response_text)  # Add the generated response as a paragraph
    doc.save(output_path)  # Save the document to the specified path


In [ ]:
# Usage example
# Prompt the user for their OpenAI API key
api_key ="Enteryou key"

# Initialize the OpenAIChat class
chat = OpenAIChat(api_key=api_key, model_name="gpt-4o")

In [ ]:
# Load DOCX data
docx_file_path = '/content/SRS (5).docx'
chat.load_docx(file_path=docx_file_path)
# Split text into chunks
chat.split_text()

# Create vector store
chat.create_vectorstore()

# Create conversation chain
chat.create_conversation_chain()
# Ask questions
# Generate Django models from SRS
query = """
I am working on a project that requires the development of a complete static website based on a Software Requirements Specification (SRS) document. The SRS outlines the necessary features, functionality, and design elements for the website. To begin the development process, I need to identify all the HTML pages that need to be created.

Please provide a detailed list of HTML pages that are required according to the SRS. For each page, include a brief description of its purpose, key features, and any specific elements or sections that should be included.:
""" + chat.data[0].page_content
code = chat.ask_question(query)
print(code)

Sure, I can help you draft a detailed list of HTML pages based on the typical requirements outlined in a Software Requirements Specification (SRS) document for an online e-commerce website. Here is a comprehensive list of HTML pages along with their descriptions, key features, and specific elements or sections:

1. **Homepage**
   - **Purpose:** Serve as the landing page for the website.
   - **Key Features:** 
     - Header section with site title, logo, and navigation menu.
     - Prominent display of the latest promotions, deals, or discounts.
     - Featured products or categories showcased above the fold.
     - Visible CTAs for creating accounts, signing up for newsletters, or making purchases.
   - **Specific Elements:** 
     - Hero banner
     - Promotional banners
     - Featured products section
     - Footer with contact information and social media links

2. **Category Page**
   - **Purpose:** Display products within a specific category.
   - **Key Features:** 
     - Brea

In [ ]:
# Store the generated Django models in a DOCX file
output_doc_path='/content/ListofHTMLPage.docx'
store_response_in_docx(code, output_doc_path)

In [ ]:
# Load DOCX data
docx_file_path = '/content/ListofHTMLPage.docx'
chat.load_docx(file_path=docx_file_path)
# Split text into chunks
chat.split_text()

# Create vector store
chat.create_vectorstore()

# Create conversation chain
chat.create_conversation_chain()
# Ask questions
# Generate Django models from SRS
query = """
Generate the complete HTML pages, CSS, and JavaScript code for a static website based on the provided document content. The website should include:
Add sections and HTML Pages  according to your own knowldged about similar project.
Ensure the HTML, CSS, and JavaScript are in separate files. Provide the complete code for each file. Here is the SRS document content:
""" + chat.data[0].page_content
code = chat.ask_question(query)
print(code)

The provided content outlines a solid foundation for a static e-commerce website. However, it currently covers only the homepage, categories, login, about, and contact pages. Here is the complete code, including the additional pages mentioned in the context: Registration, FAQ, Terms and Conditions, Privacy Policy, and Error Page (404).

### HTML: index.html
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="description" content="An online e-commerce website built using Django platform">
    <meta name="keywords" content="e-commerce, online shopping, Django">
    <meta name="author" content="Your Name">
    <title>Online E-commerce Website</title>
    <link rel="stylesheet" href="styles.css">
</head>
<body>
    <header>
        <nav>
            <ul>
                <li><a href="index.html">Home</a></li>
                <li><a href="categories.html">Categories</a></li>
    

In [ ]:
# Store the generated Django models in a DOCX file
output_doc_path='/content/HTMLS.docx'
store_response_in_docx(code, output_doc_path)

In [ ]:
import os
import re
from docx import Document

class CodeExtractor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.text = self._extract_text_from_docx()

    def _extract_text_from_docx(self):
        doc = Document(self.file_path)
        full_text = []
        for paragraph in doc.paragraphs:
            full_text.append(paragraph.text)
        return '\n'.join(full_text)

    def _extract_code_blocks(self, language):
        pattern = re.compile(f'```{language}(.*?)```', re.DOTALL)
        return pattern.findall(self.text)

    def _extract_file_name(self, block, extension):
        # Extract the filename from the content inside the code block
        title_pattern = re.compile(r'<title>(.*?)</title>', re.DOTALL)
        match = title_pattern.search(block)
        if match:
            title = match.group(1).strip().replace(' ', '_').lower()
            return f"{title}.{extension}"
        return f"default_{extension}.{extension}"

    def save_files(self, blocks, directory, extension):
        os.makedirs(directory, exist_ok=True)
        for block in blocks:
            file_name = self._extract_file_name(block, extension)
            file_path = os.path.join(directory, file_name)
            with open(file_path, 'w') as file:
                file.write(block.strip())

    def extract_and_save_all(self):
        languages = {'html': 'html_files', 'css': 'css_files', 'js': 'js_files'}
        for language, directory in languages.items():
            blocks = self._extract_code_blocks(language)
            self.save_files(blocks, directory, language)

if __name__ == '__main__':
    file_path = '/content/HTMLS.docx'
    extractor = CodeExtractor(file_path)
    extractor.extract_and_save_all()
    print("Files have been saved successfully.")


Files have been saved successfully.


In [ ]:
# Load DOCX data
docx_file_path = '/content/SRS (5).docx'
chat.load_docx(file_path=docx_file_path)
print(chat.data)
-+.
# Split text into chunks
chat.split_text()

# Create vector store
chat.create_vectorstore()

# Create conversation chain
chat.create_conversation_chain()
# Ask questions
# Generate Django models from SRS
query = "Generate all detailed Django models based on the following SRS document content use your own knowldged related to this project too:\n\n" + chat.data[0].page_content
django_models = chat.ask_question(query)
print(django_models)

Software Requirement Specification  Document 
Operating environment
Sure, I'm happy to help! For the Online Ecommerce Website project on Django Platform, the following is the detailed operating environment:
     • Operating System: Linux (either Ubuntu or CentOS)
     • Web Server: Nginx
     • Database: MySQL or PostgreSQL
     • Language: Python (Django framework specifically)
     • Framework: Django 3.x
     • Libraries: Django REST framework, Django Authentication module, Django Debug Toolbar
     • Dependencies: Pip, Git, Docker (for containerization)
     • IDE/Text Editor: Visual Studio Code, Atom, Sublime Text
     • Version Control: Git
     • Hosting Provider: AWS, Heroku, Google Cloud, Microsoft Azure

Please let me know if there's anything else I can assist you with!
Functional requirements
Of course! Here are the functional requirements for each functional area of the system for the online e commerce website built using Django platform based on the provided document:

Sea

In [ ]:
# Store the generated Django models in a DOCX file
output_doc_path='/content/Model.docx'
store_response_in_docx(django_models, output_doc_path)

In [ ]:
# Load DOCX data
docx_file_path = '/content/Model.docx'
chat.load_docx(file_path=docx_file_path)

# Split text into chunks
chat.split_text()

# Create vector store
chat.create_vectorstore()

# Create conversation chain
chat.create_conversation_chain()
# Ask questions
# Generate Django models from SRS
query = "Generate Django form classes for the following Document I want Djano Forms classes only. Use your own knowledge related to this project too:\n\n"  + chat.data[0].page_content
django_forms = chat.ask_question(query)
print(django_models)
# Store the generated Django models in a DOCX file
output_doc_path='/content/Forms.docx'
store_response_in_docx(django_models, output_doc_path)

Based on the provided Software Requirement Specification (SRS) document for the Online Ecommerce Website project using Django, here's a comprehensive set of Django models that would cover the functional areas typically involved in such a project.

```python
from django.db import models
from django.contrib.auth.models import User

# Category Model
class Category(models.Model):
    name = models.CharField(max_length=255)
    description = models.TextField(blank=True)
    slug = models.SlugField(max_length=255, unique=True)
    
    def __str__(self):
        return self.name

# Product Model
class Product(models.Model):
    category = models.ForeignKey(Category, related_name='products', on_delete=models.CASCADE)
    name = models.CharField(max_length=255)
    description = models.TextField()
    slug = models.SlugField(max_length=255, unique=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    stock = models.PositiveIntegerField()
    available = models.BooleanField

In [ ]:
# Load DOCX data
docx_file_path = '/content/Forms.docx'
chat.load_docx(file_path=docx_file_path)

# Split text into chunks
chat.split_text()

# Create vector store
chat.create_vectorstore()

# Create conversation chain
chat.create_conversation_chain()
# Ask questions
# Generate Django models from SRS
query = f"Generate all Django forms class based on provided django models document content use your own knowldged related to this project too:\n\n" + chat.data[0].page_content
django_forms = chat.ask_question(query)
print(django_models)
# Store the generated Django models in a DOCX file
output_doc_path='/content/Views.docx'
store_response_in_docx(django_models, output_doc_path)

In [ ]:
# Load DOCX data
docx_file_path = '/content/Views.docx'
chat.load_docx(file_path=docx_file_path)

# Split text into chunks
chat.split_text()

# Create vector store
chat.create_vectorstore()

# Create conversation chain
chat.create_conversation_chain()
# Ask questions
# Generate Django models from SRS
query = f"Generate all Django forms class based on provided django models document content use your own knowldged related to this project too:\n\n" + chat.data[0].page_content
django_forms = chat.ask_question(query)
print(django_models)
# Store the generated Django models in a DOCX file
output_doc_path='/content/HTML.docx'
store_response_in_docx(django_models, output_doc_path)

To generate Django models based on the provided Software Requirement Specification (SRS) document for an online e-commerce website, I'll create models that fit common e-commerce features, including products, categories, users, orders, and payments. These models will be designed to align with the functional requirements described and typical e-commerce application needs.

Here is a sample implementation of the Django models:

```python
from django.db import models
from django.contrib.auth.models import User

class Category(models.Model):
    name = models.CharField(max_length=255)
    description = models.TextField(blank=True, null=True)

    def __str__(self):
        return self.name

class Product(models.Model):
    name = models.CharField(max_length=255)
    description = models.TextField()
    price = models.DecimalField(max_digits=10, decimal_places=2)
    stock = models.IntegerField()
    category = models.ForeignKey(Category, related_name='products', on_delete=models.CASCADE)
  

In [ ]:
query = f"Generate all Django views based django models {django_models} on the following SRS document content use your own knowldged related to this project too:\n\n" + chat.data[0].page_content
django_forms = chat.ask_question(query)
print(django_models)